In [272]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import os
import re

### Load base data of matches from http://tennis-data.co.uk

In [274]:
# Load file names to array

data_folder = './data/base_matches'
data_format = 'xlsx'

xls_files = []

for file in os.listdir(data_folder):
    if file.endswith(data_format):
        xls_files.append(file)
print(f'Chosen files: {xls_files}')

Chosen files: ['2013.xlsx', '2014.xlsx', '2015.xlsx', '2016.xlsx', '2017.xlsx', '2018.xlsx', '2019.xlsx', '2020.xlsx', '2021.xlsx', '2022.xlsx', '2023.xlsx']


In [275]:
# Create dataframe for each file
dataframes = {}
for xls_file in xls_files:
    file_path = os.path.join(data_folder, xls_file)
    df = pd.read_excel(file_path, engine='openpyxl')
    dataframes[xls_file] = df
    
for df_name in dataframes:
    head = dataframes[df_name].shape
    print(f'{df_name}:\n {head}')

D:\annaconda\lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
D:\annaconda\lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
D:\annaconda\lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
D:\annaconda\lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


2013.xlsx:
 (2631, 42)
2014.xlsx:
 (2600, 42)
2015.xlsx:
 (2630, 40)
2016.xlsx:
 (2626, 40)
2017.xlsx:
 (2633, 40)
2018.xlsx:
 (2637, 40)
2019.xlsx:
 (2610, 36)
2020.xlsx:
 (1267, 36)
2021.xlsx:
 (2489, 36)
2022.xlsx:
 (2632, 36)
2023.xlsx:
 (1101, 36)


In [276]:
# concat all dataframes into one
df = pd.concat(dataframes.values(), axis=0)

In [277]:
print(df.shape)

(25856, 42)


### Save combined dfs

In [278]:
df.to_csv(f'{data_folder}/combined_shuffled_{xls_files[0].split(".")[0]}_to_{xls_files[-1].split(".")[0]}.csv')

### Shuffle the data and create player_1, player_2, player_1_won  columns

#### Player_1_won 1 if player_1 wins, player_1_won 0 if player_1 looses

In [279]:
# Extract players from Winner and Looser fields to player_1, player_2 and shuffle them
# Shuffle players
shuffled_df = df.copy()
shuffled_df['player_1_won'] = 1

shuffled_df['swap'] = np.random.rand(len(shuffled_df)) > 0.5
shuffled_df.loc[shuffled_df['swap'], 'player_1_won'] = 0

shuffled_df['player_1'] = np.where(shuffled_df['swap'], shuffled_df['Loser'], shuffled_df['Winner'])
shuffled_df['player_2'] = np.where(shuffled_df['swap'], shuffled_df['Winner'], shuffled_df['Loser'])

### ADD RANK INFO FOR EACH PLAYER

In [280]:
shuffled_df['player_1_rank'] = np.where(shuffled_df['player_1_won'] == 0, shuffled_df['LRank'], shuffled_df['WRank'])
shuffled_df['player_2_rank'] = np.where(shuffled_df['player_1_won'] == 0, shuffled_df['WRank'], shuffled_df['LRank'])

shuffled_df['player_1_points'] = np.where(shuffled_df['player_1_won'] == 0, shuffled_df['LPts'], shuffled_df['WPts'])
shuffled_df['player_2_points'] = np.where(shuffled_df['player_1_won'] == 0, shuffled_df['WPts'], shuffled_df['LPts'])

### Add odds for each player

In [281]:
shuffled_df['player_1_odds'] = 0.0
shuffled_df['player_2_odds'] = 0.0
shuffled_df['player_1_odds'] = np.where(shuffled_df['swap'], shuffled_df['B365L'], shuffled_df['B365W'])
shuffled_df['player_2_odds'] = np.where(shuffled_df['swap'], shuffled_df['B365W'], shuffled_df['B365L'])

In [282]:
shuffled_df.head()

,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Winner,...,player_1_won,swap,player_1,player_2,player_1_rank,player_2_rank,player_1_points,player_2_points,player_1_odds,player_2_odds
0,1,Brisbane,Brisbane International,2012-12-31,ATP250,Outdoor,Hard,1st Round,3.0,Mayer F.,...,1,False,Mayer F.,Giraldo S.,28.0,57.0,1215.0,778.0,1.36,3.00
1,1,Brisbane,Brisbane International,2012-12-31,ATP250,Outdoor,Hard,1st Round,3.0,Nieminen J.,...,1,False,Nieminen J.,Benneteau J.,41.0,35.0,927.0,1075.0,1.61,2.20
2,1,Brisbane,Brisbane International,2012-12-31,ATP250,Outdoor,Hard,1st Round,3.0,Nishikori K.,...,1,False,Nishikori K.,Matosevic M.,19.0,49.0,1830.0,845.0,1.25,3.75
3,1,Brisbane,Brisbane International,2012-12-31,ATP250,Outdoor,Hard,1st Round,3.0,Baghdatis M.,...,0,True,Mitchell B.,Baghdatis M.,326.0,36.0,137.0,1070.0,9.00,1.07
4,1,Brisbane,Brisbane International,2013-01-01,ATP250,Outdoor,Hard,1st Round,3.0,Istomin D.,...,0,True,Klizan M.,Istomin D.,30.0,43.0,1175.0,897.0,1.80,1.90


In [283]:
shuffled_df.to_csv(f'{data_folder}/combined_{xls_files[0].split(".")[0]}_to_{xls_files[-1].split(".")[0]}.csv', index=False)

### Add point difference as absolute value

In [284]:
enhanced_df = shuffled_df.copy()

In [285]:

# Add rank and point difference in absolute value
rankDifference = abs(enhanced_df['player_1_rank'] - enhanced_df['player_2_rank'])
pointsDifference = abs(enhanced_df['player_1_points'] - enhanced_df['player_2_points'])
enhanced_df['rank_diff_abs'] = rankDifference
enhanced_df['point_diff_abs'] = pointsDifference
enhanced_df.head()
# cleaned_df.head

,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Winner,...,player_1,player_2,player_1_rank,player_2_rank,player_1_points,player_2_points,player_1_odds,player_2_odds,rank_diff_abs,point_diff_abs
0,1,Brisbane,Brisbane International,2012-12-31,ATP250,Outdoor,Hard,1st Round,3.0,Mayer F.,...,Mayer F.,Giraldo S.,28.0,57.0,1215.0,778.0,1.36,3.00,29.0,437.0
1,1,Brisbane,Brisbane International,2012-12-31,ATP250,Outdoor,Hard,1st Round,3.0,Nieminen J.,...,Nieminen J.,Benneteau J.,41.0,35.0,927.0,1075.0,1.61,2.20,6.0,148.0
2,1,Brisbane,Brisbane International,2012-12-31,ATP250,Outdoor,Hard,1st Round,3.0,Nishikori K.,...,Nishikori K.,Matosevic M.,19.0,49.0,1830.0,845.0,1.25,3.75,30.0,985.0
3,1,Brisbane,Brisbane International,2012-12-31,ATP250,Outdoor,Hard,1st Round,3.0,Baghdatis M.,...,Mitchell B.,Baghdatis M.,326.0,36.0,137.0,1070.0,9.00,1.07,290.0,933.0
4,1,Brisbane,Brisbane International,2013-01-01,ATP250,Outdoor,Hard,1st Round,3.0,Istomin D.,...,Klizan M.,Istomin D.,30.0,43.0,1175.0,897.0,1.80,1.90,13.0,278.0


### Point difference for each player

In [286]:
p1_rankDifference = enhanced_df['player_1_rank'] - enhanced_df['player_2_rank']
p1_pointsDifference = enhanced_df['player_1_points'] - enhanced_df['player_2_points']
p2_rankDifference = enhanced_df['player_2_rank'] - enhanced_df['player_1_rank']
p2_pointsDifference = enhanced_df['player_2_points'] - enhanced_df['player_1_points']
enhanced_df['player_1_rank_diff'] = p1_rankDifference
enhanced_df['player_2_rank_diff'] = p2_rankDifference

enhanced_df['player_1_point_diff'] = p1_pointsDifference
enhanced_df['player_2_point_diff'] = p2_pointsDifference

enhanced_df.head()

,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Winner,...,player_1_points,player_2_points,player_1_odds,player_2_odds,rank_diff_abs,point_diff_abs,player_1_rank_diff,player_2_rank_diff,player_1_point_diff,player_2_point_diff
0,1,Brisbane,Brisbane International,2012-12-31,ATP250,Outdoor,Hard,1st Round,3.0,Mayer F.,...,1215.0,778.0,1.36,3.00,29.0,437.0,-29.0,29.0,437.0,-437.0
1,1,Brisbane,Brisbane International,2012-12-31,ATP250,Outdoor,Hard,1st Round,3.0,Nieminen J.,...,927.0,1075.0,1.61,2.20,6.0,148.0,6.0,-6.0,-148.0,148.0
2,1,Brisbane,Brisbane International,2012-12-31,ATP250,Outdoor,Hard,1st Round,3.0,Nishikori K.,...,1830.0,845.0,1.25,3.75,30.0,985.0,-30.0,30.0,985.0,-985.0
3,1,Brisbane,Brisbane International,2012-12-31,ATP250,Outdoor,Hard,1st Round,3.0,Baghdatis M.,...,137.0,1070.0,9.00,1.07,290.0,933.0,290.0,-290.0,-933.0,933.0
4,1,Brisbane,Brisbane International,2013-01-01,ATP250,Outdoor,Hard,1st Round,3.0,Istomin D.,...,1175.0,897.0,1.80,1.90,13.0,278.0,-13.0,13.0,278.0,-278.0


### Calculate ELOS and Elos Differences

In [287]:
INITIAL_ELO = 1500
elo_ratings = {}
K_FACTOR = 32

def expected_outcome(rating_a, rating_b):
    expected_a = 1 / (1 + 10 ** ((rating_b - rating_a) / 400))
    expected_b = 1 - expected_a
    return expected_a, expected_b

def update_elo(player_a, player_b, player_a_won, k_factor):
    rating_a = elo_ratings.get(player_a, INITIAL_ELO)
    rating_b = elo_ratings.get(player_b, INITIAL_ELO)

    expected_a, expected_b = expected_outcome(rating_a, rating_b)

    actual_a = 1 if player_a_won else 0
    actual_b = 1 - actual_a

    new_rating_a = rating_a + k_factor * (actual_a - expected_a)
    new_rating_b = rating_b + k_factor * (actual_b - expected_b)

    elo_ratings[player_a] = new_rating_a
    elo_ratings[player_b] = new_rating_b

    return rating_a, rating_b

player_1_pre_elo = []
player_2_pre_elo = []

# Iterate through each row in the DataFrame
for index, row in enhanced_df.iterrows():
    
    player_1 = row['player_1']
    player_2 = row['player_2']

    player_a_won = row['player_1_won']

    # Update Elo ratings after the match
    pre_rating_a, pre_rating_b = update_elo(player_1, player_2, player_a_won, K_FACTOR)
    
    player_1_pre_elo.append(pre_rating_a)
    player_2_pre_elo.append(pre_rating_b)

enhanced_df['player_1_elo'] = player_1_pre_elo
enhanced_df['player_2_elo'] = player_2_pre_elo


In [288]:
enhanced_df.tail(10)

,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Winner,...,player_1_odds,player_2_odds,rank_diff_abs,point_diff_abs,player_1_rank_diff,player_2_rank_diff,player_1_point_diff,player_2_point_diff,player_1_elo,player_2_elo
1091,27,Madrid,Mutua Madrid Open,2023-05-02,Masters 1000,Outdoor,Clay,4th Round,3.0,Zhang Zh.,...,1.17,5.00,89.0,2684.0,-89.0,89.0,2684.0,-2684.0,1890.709125,1530.726615
1092,27,Madrid,Mutua Madrid Open,2023-05-02,Masters 1000,Outdoor,Clay,4th Round,3.0,Struff J.L.,...,2.30,1.62,2.0,12.0,2.0,-2.0,-12.0,12.0,1547.259111,1615.919530
1093,27,Madrid,Mutua Madrid Open,2023-05-02,Masters 1000,Outdoor,Clay,4th Round,3.0,Tsitsipas S.,...,1.14,5.50,37.0,4237.0,-37.0,37.0,4237.0,-4237.0,1905.095032,1614.697009
1094,27,Madrid,Mutua Madrid Open,2023-05-03,Masters 1000,Outdoor,Clay,Quarterfinals,3.0,Alcaraz C.,...,1.07,9.00,10.0,3915.0,-10.0,10.0,3915.0,-3915.0,2055.601297,1842.714924
1095,27,Madrid,Mutua Madrid Open,2023-05-03,Masters 1000,Outdoor,Clay,Quarterfinals,3.0,Coric B.,...,1.50,2.63,72.0,1257.0,-72.0,72.0,1257.0,-1257.0,1707.064179,1447.540767
1096,27,Madrid,Mutua Madrid Open,2023-05-04,Masters 1000,Outdoor,Clay,Quarterfinals,3.0,Karatsev A.,...,1.57,2.38,22.0,109.0,22.0,-22.0,-109.0,109.0,1580.496888,1559.148190
1097,27,Madrid,Mutua Madrid Open,2023-05-04,Masters 1000,Outdoor,Clay,Quarterfinals,3.0,Struff J.L.,...,1.20,4.50,60.0,4416.0,-60.0,60.0,4416.0,-4416.0,1910.157505,1628.798130
1098,27,Madrid,Mutua Madrid Open,2023-05-05,Masters 1000,Outdoor,Clay,Semifinals,3.0,Alcaraz C.,...,10.00,1.06,18.0,4880.0,18.0,-18.0,-4880.0,4880.0,1712.930789,2062.864492
1099,27,Madrid,Mutua Madrid Open,2023-05-05,Masters 1000,Outdoor,Clay,Semifinals,3.0,Struff J.L.,...,2.00,1.80,56.0,282.0,-56.0,56.0,282.0,-282.0,1655.509962,1595.514980
1100,27,Madrid,Mutua Madrid Open,2023-05-07,Masters 1000,Outdoor,Clay,The Final,3.0,Alcaraz C.,...,1.06,10.00,63.0,5991.0,-63.0,63.0,5991.0,-5991.0,2066.630934,1668.774228


In [289]:
p1_elo_diff = enhanced_df['player_1_elo'] - enhanced_df['player_2_elo']
p2_elo_diff = enhanced_df['player_2_elo'] - enhanced_df['player_1_elo']

enhanced_df['player_1_elo_diff'] = p1_elo_diff
enhanced_df['player_2_elo_diff'] = p2_elo_diff

elo_diff_abs = abs(enhanced_df['player_2_elo'] - enhanced_df['player_1_elo'])
enhanced_df['elo_diff_abs'] = elo_diff_abs

enhanced_df.shape

(25856, 63)

### Add Head to Head wins loss stats

In [290]:
# Initialize dictionaries to store head-to-head wins and losses
head_to_head_wins = {}
head_to_head_losses = {}

# Define a function to retrieve head-to-head win counts
def get_head_to_head_wins(player_1, player_2):
    return head_to_head_wins.get(player_1, {}).get(player_2, 0)

# Define a function to retrieve head-to-head loss counts
def get_head_to_head_losses(player_1, player_2):
    return head_to_head_losses.get(player_1, {}).get(player_2, 0)


# Initialize lists to store the head-to-head win and loss counts before each match
player_1_vs_player_2_win_pre = []
player_2_vs_player_1_win_pre = []
player_1_vs_player_2_loss_pre = []
player_2_vs_player_1_loss_pre = []

# Iterate through each row in the DataFrame
for index, row in enhanced_df.iterrows():
    player_1_won = row['player_1_won']
    player_1 = row['player_1']
    player_2 = row['player_2']

    player_a_won = player_1_won

    # Store pre-match head-to-head win and loss counts
    player_1_vs_player_2_win_pre.append(get_head_to_head_wins(player_1, player_2))
    player_2_vs_player_1_win_pre.append(get_head_to_head_wins(player_2, player_1))
    player_1_vs_player_2_loss_pre.append(get_head_to_head_losses(player_1, player_2))
    player_2_vs_player_1_loss_pre.append(get_head_to_head_losses(player_2, player_1))

    # Update head-to-head wins and losses after the match
    if player_a_won:
        # Update head-to-head wins
        if player_1 not in head_to_head_wins:
            head_to_head_wins[player_1] = {}
        head_to_head_wins[player_1][player_2] = head_to_head_wins[player_1].get(player_2, 0) + 1

        # Update head-to-head losses
        if player_2 not in head_to_head_losses:
            head_to_head_losses[player_2] = {}
        head_to_head_losses[player_2][player_1] = head_to_head_losses[player_2].get(player_1, 0) + 1

    else:
        # Update head-to-head wins
        if player_2 not in head_to_head_wins:
            head_to_head_wins[player_2] = {}
        head_to_head_wins[player_2][player_1] = head_to_head_wins[player_2].get(player_1, 0) + 1

        # Update head-to-head losses
        if player_1 not in head_to_head_losses:
            head_to_head_losses[player_1] = {}
        head_to_head_losses[player_1][player_2] = head_to_head_losses[player_1].get(player_2, 0) + 1

    # Update Elo ratings after the match
    update_elo(player_1, player_2, player_a_won, K_FACTOR)

# Add the pre-match head-to-head win and loss counts as new columns in the DataFrame
enhanced_df['player_1_vs_player_2_win'] = player_1_vs_player_2_win_pre
enhanced_df['player_2_vs_player_1_win'] = player_2_vs_player_1_win_pre
enhanced_df['player_1_vs_player_2_loss'] = player_1_vs_player_2_loss_pre
enhanced_df['player_2_vs_player_1_loss'] = player_2_vs_player_1_loss_pre

# Print the enhanced DataFrame
print(enhanced_df.head())

   ATP  Location              Tournament       Date  Series    Court Surface  \
0    1  Brisbane  Brisbane International 2012-12-31  ATP250  Outdoor    Hard   
1    1  Brisbane  Brisbane International 2012-12-31  ATP250  Outdoor    Hard   
2    1  Brisbane  Brisbane International 2012-12-31  ATP250  Outdoor    Hard   
3    1  Brisbane  Brisbane International 2012-12-31  ATP250  Outdoor    Hard   
4    1  Brisbane  Brisbane International 2013-01-01  ATP250  Outdoor    Hard   

       Round  Best of        Winner  ... player_2_point_diff  player_1_elo  \
0  1st Round      3.0      Mayer F.  ...              -437.0        1500.0   
1  1st Round      3.0   Nieminen J.  ...               148.0        1500.0   
2  1st Round      3.0  Nishikori K.  ...              -985.0        1500.0   
3  1st Round      3.0  Baghdatis M.  ...               933.0        1500.0   
4  1st Round      3.0    Istomin D.  ...              -278.0        1500.0   

   player_2_elo  player_1_elo_diff  player_2_elo_d

In [291]:
# Save DF
enhanced_df.to_csv('./data/combined_shuffled_enhanced.csv', index=False)

### ADD COMBINE DF WITH DATA FROM https://tennisabstract.com/reports/atp_elo_ratings.html to get the ELO of surface and current age

In [292]:
tenisabstract_df = pd.read_csv('./data/elos/tennis_abstract.csv')

In [293]:
tenisabstract_df.shape

(448, 12)

### Convert the names to match the original DF

In [294]:
def convert_name_format(name):
    # Split the name into a list of words
    name_parts = name.split()
    
    # Extract the first name and last name
    first_name = name_parts[0]
    last_name = name_parts[-1]
    
    # Create the abbreviated format: "Last name first initial."
    abbreviated_name = f"{last_name} {first_name[0]}."
    
    return abbreviated_name

In [295]:
tenisabstract_df['converted_name'] = tenisabstract_df['Player'].apply(convert_name_format)


### Fix column names, remove spaces replace with _

In [296]:
tenisabstract_df.columns = [re.sub(r'\s+', '_', col.lower()) for col in tenisabstract_df.columns]

# Print the updated column names
print(tenisabstract_df.columns)

Index(['player', 'age', 'elo', 'hardraw', 'clayraw', 'grassraw', 'helo',
       'celo', 'gelo', 'peak_match', 'peak_age', 'peak_elo', 'converted_name'],
      dtype='object')


In [297]:
### Fix Empty Values  - to NaN

# Replace standalone '-' values with NaN
tenisabstract_df = tenisabstract_df.replace(r'^-$', np.nan, regex=True)

# Print the updated DataFrame
print(tenisabstract_df)

                      player   age     elo hardraw clayraw grassraw    helo  \
0             Carlos Alcaraz  19.9  2159.5    2037  2014.7     1500  2098.2   
1             Novak Djokovic  35.9  2151.9  2075.4  1891.4   1871.6  2113.7   
2            Daniil Medvedev  27.1  2132.8  2060.8  1730.7   1731.8  2096.8   
3              Jannik Sinner  21.6  2101.4  1996.4  1965.4   1543.3  2048.9   
4         Stefanos Tsitsipas  24.6  2047.0  1922.4  2017.8     1711  1984.7   
..                       ...   ...     ...     ...     ...      ...     ...   
443  Conner Huertas Del Pino  27.1  1165.8     NaN  1148.4      NaN  1192.9   
444   Rodrigo Pacheco Mendez  17.9  1142.9  1144.1    1090      NaN  1143.5   
445             Wilson Leite  31.4  1112.3  1060.8  1112.2      NaN  1086.6   
446            Daniel Rincon  20.2  1109.6  1110.1  1090.2   1323.7  1109.8   
447           Naoki Nakagawa  26.4  1072.0  1075.5  1027.7      NaN  1073.8   

       celo    gelo                  peak_match  pe

## Combine the lines by converted_name as FK to the other DF

In [298]:
# Merge the DataFrames on player_1 and player_2 names
merged_df_player_1 = enhanced_df.merge(tenisabstract_df, left_on='player_1', right_on='converted_name', how='left')
merged_df_both_players = merged_df_player_1.merge(tenisabstract_df, left_on='player_2', right_on='converted_name', how='left', suffixes=('_player_1', '_player_2'))

# Print the merged DataFrame with both player_1 and player_2 information
print(merged_df_both_players)
df = merged_df_both_players.copy()

       ATP  Location              Tournament       Date        Series  \
0        1  Brisbane  Brisbane International 2012-12-31        ATP250   
1        1  Brisbane  Brisbane International 2012-12-31        ATP250   
2        1  Brisbane  Brisbane International 2012-12-31        ATP250   
3        1  Brisbane  Brisbane International 2012-12-31        ATP250   
4        1  Brisbane  Brisbane International 2013-01-01        ATP250   
...    ...       ...                     ...        ...           ...   
25851   27    Madrid       Mutua Madrid Open 2023-05-04  Masters 1000   
25852   27    Madrid       Mutua Madrid Open 2023-05-04  Masters 1000   
25853   27    Madrid       Mutua Madrid Open 2023-05-05  Masters 1000   
25854   27    Madrid       Mutua Madrid Open 2023-05-05  Masters 1000   
25855   27    Madrid       Mutua Madrid Open 2023-05-07  Masters 1000   

         Court Surface          Round  Best of        Winner  ...  \
0      Outdoor    Hard      1st Round      3.0      Ma

In [299]:
# Save the progress
df.to_csv('./data/merged.csv', index = False)

In [300]:
bookie_wrong = []

for index, row in df.iterrows():
    player_1_won = row['player_1_won']
    player_1_odds = row['player_1_odds']
    player_2_odds = row['player_2_odds']
    if player_1_won:
        if player_1_odds >= player_2_odds:
            bookie_wrong.append(1)
        else:
            bookie_wrong.append(0)
    else:
        if player_2_odds >= player_1_odds:
            bookie_wrong.append(1)
        else:
            bookie_wrong.append(0)
            
df['bookie_wrong'] = bookie_wrong
            

print(df.head())

   ATP  Location              Tournament       Date  Series    Court Surface  \
0    1  Brisbane  Brisbane International 2012-12-31  ATP250  Outdoor    Hard   
1    1  Brisbane  Brisbane International 2012-12-31  ATP250  Outdoor    Hard   
2    1  Brisbane  Brisbane International 2012-12-31  ATP250  Outdoor    Hard   
3    1  Brisbane  Brisbane International 2012-12-31  ATP250  Outdoor    Hard   
4    1  Brisbane  Brisbane International 2013-01-01  ATP250  Outdoor    Hard   

       Round  Best of        Winner  ... clayraw_player_2  grassraw_player_2  \
0  1st Round      3.0      Mayer F.  ...              NaN                NaN   
1  1st Round      3.0   Nieminen J.  ...              NaN                NaN   
2  1st Round      3.0  Nishikori K.  ...              NaN                NaN   
3  1st Round      3.0  Baghdatis M.  ...              NaN                NaN   
4  1st Round      3.0    Istomin D.  ...           1387.6             1433.4   

   helo_player_2  celo_player_2  gelo_

In [304]:
df.head(20)

,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Winner,...,clayraw_player_2,grassraw_player_2,helo_player_2,celo_player_2,gelo_player_2,peak_match_player_2,peak_age_player_2,peak_elo_player_2,converted_name_player_2,bookie_wrong
0,1,Brisbane,Brisbane International,2012-12-31,ATP250,Outdoor,Hard,1st Round,3.0,Mayer F.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,1,Brisbane,Brisbane International,2012-12-31,ATP250,Outdoor,Hard,1st Round,3.0,Nieminen J.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,1,Brisbane,Brisbane International,2012-12-31,ATP250,Outdoor,Hard,1st Round,3.0,Nishikori K.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,1,Brisbane,Brisbane International,2012-12-31,ATP250,Outdoor,Hard,1st Round,3.0,Baghdatis M.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,1,Brisbane,Brisbane International,2013-01-01,ATP250,Outdoor,Hard,1st Round,3.0,Istomin D.,...,1387.6,1433.4,1414.2,1405.9,1428.8,2012 Indian Wells R32,25.4,1849.2,Istomin D.,1
5,1,Brisbane,Brisbane International,2013-01-01,ATP250,Outdoor,Hard,1st Round,3.0,Millman J.,...,1540,1484.1,1631.6,1605.6,1577.6,2018 US Open R16,29.2,1842.8,Millman J.,0
6,1,Brisbane,Brisbane International,2013-01-01,ATP250,Outdoor,Hard,1st Round,3.0,Falla A.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
7,1,Brisbane,Brisbane International,2013-01-01,ATP250,Outdoor,Hard,1st Round,3.0,Melzer J.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
8,1,Brisbane,Brisbane International,2013-01-01,ATP250,Outdoor,Hard,1st Round,3.0,Robredo T.,...,1478.9,1367.5,1500.2,1508.2,1452.4,2017 Acapulco Qual,24.8,1824.2,Harrison R.,1
9,1,Brisbane,Brisbane International,2013-01-01,ATP250,Outdoor,Hard,1st Round,3.0,Dimitrov G.,...,1771.5,1517.5,1851.0,1837.3,1710.3,2014 Canada QF,23.2,2090.5,Dimitrov G.,0


In [302]:
df.columns

Index(['ATP', 'Location', 'Tournament', 'Date', 'Series', 'Court', 'Surface',
       'Round', 'Best of', 'Winner', 'Loser', 'WRank', 'LRank', 'WPts', 'LPts',
       'W1', 'L1', 'W2', 'L2', 'W3', 'L3', 'W4', 'L4', 'W5', 'L5', 'Wsets',
       'Lsets', 'Comment', 'B365W', 'B365L', 'EXW', 'EXL', 'LBW', 'LBL', 'PSW',
       'PSL', 'SJW', 'SJL', 'MaxW', 'MaxL', 'AvgW', 'AvgL', 'player_1_won',
       'swap', 'player_1', 'player_2', 'player_1_rank', 'player_2_rank',
       'player_1_points', 'player_2_points', 'player_1_odds', 'player_2_odds',
       'rank_diff_abs', 'point_diff_abs', 'player_1_rank_diff',
       'player_2_rank_diff', 'player_1_point_diff', 'player_2_point_diff',
       'player_1_elo', 'player_2_elo', 'player_1_elo_diff',
       'player_2_elo_diff', 'elo_diff_abs', 'player_1_vs_player_2_win',
       'player_2_vs_player_1_win', 'player_1_vs_player_2_loss',
       'player_2_vs_player_1_loss', 'player_player_1', 'age_player_1',
       'elo_player_1', 'hardraw_player_1', 'clayraw_

In [210]:
df = pd.read_csv('./exports/combined_data_cleared_enhanced2.csv')

### Count wins

In [ ]:
wins = {}
losses = {}


# Iterate through each row in the DataFrame
for index, row in df.iterrows():
    player_1_won = not row['player_1_won']
    player_1 = row['player_1']
    player_2 = row['player_2']

    player_a_won = player_1_won

    if player_a_won:
        wins[player_1] = wins.get(player_1, 0) + 1
        losses[player_2] = losses.get(player_2, 0) + 1
    else:
        wins[player_2] = wins.get(player_2, 0) + 1
        losses[player_1] = losses.get(player_1, 0) + 1

# Add wins and losses to the DataFrame
df['player_1_wins'] = df['player_1'].map(wins).fillna(0)
df['player_1_losses'] = df['player_1'].map(losses).fillna(0)
df['player_2_wins'] = df['player_2'].map(wins).fillna(0)
df['player_2_losses'] = df['player_2'].map(losses).fillna(0)

# Print the enhanced DataFrame
print(df.head(30))

### Calc elo for players

In [ ]:
INITIAL_ELO = 1500
elo_ratings = {}
K_FACTOR = 32

def expected_outcome(rating_a, rating_b):
    expected_a = 1 / (1 + 10 ** ((rating_b - rating_a) / 400))
    expected_b = 1 - expected_a
    return expected_a, expected_b

def update_elo(player_a, player_b, player_a_won, k_factor):
    rating_a = elo_ratings.get(player_a, INITIAL_ELO)
    rating_b = elo_ratings.get(player_b, INITIAL_ELO)

    expected_a, expected_b = expected_outcome(rating_a, rating_b)

    actual_a = 1 if player_a_won else 0
    actual_b = 1 - actual_a

    new_rating_a = rating_a + k_factor * (actual_a - expected_a)
    new_rating_b = rating_b + k_factor * (actual_b - expected_b)

    elo_ratings[player_a] = new_rating_a
    elo_ratings[player_b] = new_rating_b

    return rating_a, rating_b


In [ ]:
player_1_pre_elo = []
player_2_pre_elo = []

# Iterate through each row in the DataFrame
for index, row in df.iterrows():
    
    player_1 = row['player_1']
    player_2 = row['player_2']

    player_a_won = not row['player_1_won']

    # Update Elo ratings after the match
    pre_rating_a, pre_rating_b = update_elo(player_1, player_2, player_a_won, K_FACTOR)
    
    player_1_pre_elo.append(pre_rating_a)
    player_2_pre_elo.append(pre_rating_b)

df['elo_1'] = player_1_pre_elo
df['elo_2'] = player_2_pre_elo

df.tail(30)


In [ ]:
df.to_csv('./exports/elo_loss_win.csv')